In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import plotly.express as px

import datetime as dt

In [2]:
covid_tracker_states_daily = "http://covidtracking.com/api/states/daily.csv"

In [3]:
data = pd.read_csv(covid_tracker_states_daily)

In [4]:
data.head()

,date,state,positive,negative,pending,hospitalized,death,total,dateChecked
0,20200323,AK,22.0,946.0,NaN,0.0,NaN,968,2020-03-23T20:00:00Z
1,20200323,AL,167.0,1665.0,NaN,NaN,0.0,1832,2020-03-23T20:00:00Z
2,20200323,AR,174.0,906.0,0.0,13.0,0.0,1080,2020-03-23T20:00:00Z
3,20200323,AS,NaN,NaN,NaN,NaN,0.0,0,2020-03-23T20:00:00Z
4,20200323,AZ,265.0,309.0,6.0,NaN,2.0,580,2020-03-23T20:00:00Z


In [5]:
data["date"] = pd.to_datetime(data["dateChecked"])
data["doy"] = data["date"].apply(lambda x: x.timetuple().tm_yday)
data["logPos"] = np.log10(data["positive"].replace(0,1))
data.sort_values(by="date",inplace=True)

In [6]:
foo = widgets.SelectMultiple(options=sorted(data["state"].unique()),
                                             value=['MD'],
                                             #rows=10,
                                             description='US State')
@interact
def plot_cases(states=foo,
               scalex=["linear","log"]):
    plt.figure(figsize=[12.5,7.5])
    for x in states:
        tmp = data.loc[data["state"] == x]
        plt.plot(tmp["date"].values,tmp["positive"].values,label=x,marker="o")  
    plt.yscale(scalex)
    plt.xticks(rotation=45)
    plt.yticks(size=15)
    plt.legend(fontsize="large")
    plt.ylabel("number of cases",size=20)

    

interactive(children=(SelectMultiple(description='US State', index=(22,), options=('AK', 'AL', 'AR', 'AS', 'AZ…

In [56]:
mostrecent = sorted(data["date"],reverse=True)[0]

tmp = data.loc[data["date"] == mostrecent][["state","positive"]].dropna()
tmp["state"].values

px.choropleth(locations=tmp["state"].values,
              locationmode="USA-states",
              color=np.log10(tmp["positive"].replace(0,1)).values,
              scope="usa")